<a href="https://colab.research.google.com/github/JackStrabala/mgmt467-analytics-portfolio/blob/main/Unit2_Lab2_PromptStudio_All_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio for AI-Assisted SQL + ML

**Date:** 2025-10-16  
**Objective:** Build and refine a complete ML pipeline for churn prediction using BigQuery — but with **Gemini-style prompts** guiding SQL generation.

You'll learn to:
- Frame SQL goals as clear prompts
- Generate, test, and debug queries with an AI assistant
- Reflect on each modeling step and your prompt design



## Task 0: Connect to BigQuery

**🎯 Goal:** Verify BigQuery access from Colab.  
**📌 Requirements:** Use `%%bigquery`, get current date and user session.

---

### 🧠 Prompt Template  
> Write a SQL query that returns CURRENT_DATE() and SESSION_USER(). I will run it with %%bigquery in Colab.

---

### 👩‍🏫 Example Prompt  
> Write a SQL query using BigQuery syntax that returns today’s date and the current session user.

---

### ✅ Expected SQL Output
```sql
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;
```

---

### 🔍 Checkpoint  
Query should return a single row with today's date and your user.


In [ ]:
%%bigquery
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;

Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-27,jackstrabala1@gmail.com



## Task 1: Prepare ML Table

**🎯 Goal:** Create a clean features table for modeling churn.  
**📌 Requirements:** Use cleaned_features as source, select relevant columns, filter rows with churn_label IS NOT NULL.

---

### 🧠 Prompt Template  
> Write a query that creates a new table with columns: [region, plan_tier, age_band, ...] and churn_label from [source_table]. Filter to rows where churn_label IS NOT NULL.

---

### 👩‍🏫 Example Prompt  
> Create a BigQuery table named churn_features from cleaned_features with selected features and where churn_label IS NOT NULL.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;
```

---

### 🔍 Checkpoint  
Table should appear in BigQuery and contain non-null labels.


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `netflix.churn_features` AS
WITH
-- 1. Usage features from watch_history
watch_feats AS (
  SELECT
    user_id,
    SUM(watch_duration_minutes) AS total_minutes_watched,
    AVG(watch_duration_minutes) AS avg_session_minutes
  FROM `netflix.watch_history`
  GROUP BY user_id
),

-- 2. Satisfaction / engagement-from-reviews features
review_feats AS (
  SELECT
    user_id,
    AVG(rating) AS avg_rating_given,
    COUNT(*)   AS num_reviews_left
  FROM `netflix.reviews`
  GROUP BY user_id
),

-- 3. User profile + churn label
user_base AS (
  SELECT
    u.user_id,

    -- region / market info
    u.country AS region,

    -- plan / monetization info
    u.subscription_plan AS plan_tier,

    -- bucket raw age into bands for modeling
    CASE
      WHEN u.age < 18 THEN 'under_18'
      WHEN u.age BETWEEN 18 AND 29 THEN '18_29'
      WHEN u.age BETWEEN 30 AND 44 THEN '30_44'
      WHEN u.age BETWEEN 45 AND 64 THEN '45_64'
      WHEN u.age >= 65 THEN '65_plus'
      ELSE 'unknown'
    END AS age_band,

    u.age,
    u.monthly_spend,

    -- create churn_label directly from active status
    CASE
      WHEN u.is_active = FALSE THEN 1  -- churned
      WHEN u.is_active = TRUE  THEN 0  -- retained
      ELSE NULL
    END AS churn_label
  FROM `netflix.users` u
)

-- 4. Final feature set (this is effectively cleaned_features)
SELECT
  b.user_id, -- Added user_id here
  b.region,
  b.plan_tier,
  b.age_band,
  w.total_minutes_watched AS total_minutes,
  r.avg_rating_given      AS avg_rating,
  b.churn_label
FROM user_base b
LEFT JOIN watch_feats w
  ON b.user_id = w.user_id
LEFT JOIN review_feats r
  ON b.user_id = r.user_id
WHERE
  b.churn_label IS NOT NULL;

Query is running:   0%|          |

""



## Task 2: Train Logistic Regression Model

**🎯 Goal:** Train a basic BQML logistic regression model.  
**📌 Requirements:** Use churn_features table, predict churn_label from features.

---

### 🧠 Prompt Template  
> Write a CREATE MODEL SQL for logistic regression using churn_label as label and [features] as inputs.

---

### 👩‍🏫 Example Prompt  
> Train a logistic regression model to predict churn_label using region, plan_tier, total_minutes, avg_rating.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `your_dataset.churn_features`;
```

---

### 🔍 Checkpoint  
Model appears in BigQuery under Models. Training completes.


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `netflix.churn_model`
OPTIONS(
  model_type='logistic_reg',
  input_label_cols=['churn_label']
) AS
SELECT
  region,
  plan_tier,
  total_minutes,
  avg_rating,
  churn_label
FROM
  `netflix.churn_features`;

Query is running:   0%|          |

""



## Task 3: Evaluate Model

**🎯 Goal:** Evaluate the logistic regression model.  
**📌 Requirements:** Use ML.EVALUATE.

---

### 🧠 Prompt Template  
> Write a query to evaluate my logistic regression model using ML.EVALUATE.

---

### 👩‍🏫 Example Prompt  
> Evaluate the churn_model using ML.EVALUATE to get accuracy, precision, recall.

---

### ✅ Expected SQL Output
```sql
SELECT * FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);
```

---

### 🔍 Checkpoint  
View performance metrics: accuracy, log_loss, precision, recall.


In [ ]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL `netflix.churn_model`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.856655,0.0,0.410758,0.514086



## Task 4: Predict Churn

**🎯 Goal:** Use ML.PREDICT to generate churn predictions.  
**📌 Requirements:** Apply model to same input table.

---

### 🧠 Prompt Template  
> Generate SQL to use ML.PREDICT on churn_model and return predictions by user_id.

---

### 👩‍🏫 Example Prompt  
> Predict churn using churn_model. Include user_id, predicted_churn_label, and prediction probability.

---

### ✅ Expected SQL Output
```sql
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
      (SELECT * FROM `your_dataset.churn_features`));
```

---

### 🔍 Checkpoint  
Inspect top churn risk users. Validate probabilities.


In [ ]:
%%bigquery
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `netflix.churn_model`,
      (SELECT * FROM `netflix.churn_features`));

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,predicted_churn_label,predicted_churn_label_probs
0,user_08194,0,"[{'label': 1, 'prob': 0.1449412348229009}, {'l..."
1,user_04649,0,"[{'label': 1, 'prob': 0.14237238175559358}, {'..."
2,user_08139,0,"[{'label': 1, 'prob': 0.13900352916348951}, {'..."
3,user_01749,0,"[{'label': 1, 'prob': 0.14228940511327534}, {'..."
4,user_00612,0,"[{'label': 1, 'prob': 0.14204950812486258}, {'..."
...,...,...,...
10295,user_07226,0,"[{'label': 1, 'prob': 0.1385859732186393}, {'l..."
10296,user_03450,0,"[{'label': 1, 'prob': 0.13802288697022774}, {'..."
10297,user_07002,0,"[{'label': 1, 'prob': 0.13381804160145946}, {'..."
10298,user_09471,0,"[{'label': 1, 'prob': 0.1373020447288701}, {'l..."



## Task 5.0: Bucket a Continuous Feature

**🎯 Goal:** Group 'total_minutes' into categories: low, medium, high.  
**📌 Requirements:** Use CASE WHEN or IF statements to create 'watch_time_bucket'.

---

### 🧠 Prompt Template  
> Write SQL that creates a new column watch_time_bucket based on total_minutes thresholds (<100, 100–300, >300).

---

### 👩‍🏫 Example Prompt  
> Create a new column watch_time_bucket with values 'low', 'medium', or 'high' based on total_minutes.

---

### 🔍 Exploration  
How does churn rate vary across these buckets?


In [ ]:
%%bigquery
SELECT
  *,
  CASE
    WHEN total_minutes < 100 THEN 'low'
    WHEN total_minutes >= 100 AND total_minutes <= 300 THEN 'medium'
    WHEN total_minutes > 300 THEN 'high'
    ELSE 'unknown'
  END AS watch_time_bucket
FROM
  `netflix.churn_features`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,region,plan_tier,age_band,total_minutes,avg_rating,churn_label,watch_time_bucket
0,user_08194,USA,Premium,18_29,326.9,NaN,0,high
1,user_04649,USA,Standard,18_29,595.3,NaN,0,high
2,user_08139,Canada,Premium+,18_29,645.0,NaN,0,high
3,user_01749,USA,Standard,18_29,644.3,NaN,0,high
4,user_00612,USA,Standard,18_29,786.1,NaN,0,high
5,user_05529,USA,Standard,18_29,1208.8,NaN,1,high
6,user_03420,USA,Premium+,18_29,529.7,NaN,0,high
7,user_03877,USA,Standard,18_29,743.5,NaN,0,high
8,user_00971,USA,Basic,18_29,523.0,NaN,0,high
9,user_03115,Canada,Premium+,18_29,515.6,NaN,0,high



## Task 5.1: Create a Binary Flag Feature

**🎯 Goal:** Add a binary column flag_binge (1 if total_minutes > 500).  
**📌 Requirements:** Use IF logic to create a binary column in SQL.

---

### 🧠 Prompt Template  
> Write a SQL query that adds flag_binge = 1 if total_minutes > 500, else 0.

---

### 👩‍🏫 Example Prompt  
> Add a binary column flag_binge to identify binge-watchers.

---

### 🔍 Exploration  
Are binge-watchers more or less likely to churn?


In [ ]:
%%bigquery
SELECT
  *,
  CASE
    WHEN total_minutes > 500 THEN 1
    ELSE 0
  END AS flag_binge
FROM
  `netflix.churn_features`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,region,plan_tier,age_band,total_minutes,avg_rating,churn_label,flag_binge
0,user_08194,USA,Premium,18_29,326.9,NaN,0,0
1,user_04649,USA,Standard,18_29,595.3,NaN,0,1
2,user_08139,Canada,Premium+,18_29,645.0,NaN,0,1
3,user_01749,USA,Standard,18_29,644.3,NaN,0,1
4,user_00612,USA,Standard,18_29,786.1,NaN,0,1
5,user_05529,USA,Standard,18_29,1208.8,NaN,1,1
6,user_03420,USA,Premium+,18_29,529.7,NaN,0,1
7,user_03877,USA,Standard,18_29,743.5,NaN,0,1
8,user_00971,USA,Basic,18_29,523.0,NaN,0,1
9,user_03115,Canada,Premium+,18_29,515.6,NaN,0,1



## Task 5.2: Create an Interaction Term

**🎯 Goal:** Create plan_region_combo by combining plan_tier and region.  
**📌 Requirements:** Use CONCAT or STRING functions.

---

### 🧠 Prompt Template  
> Generate SQL to create a new column by combining plan_tier and region with an underscore.

---

### 👩‍🏫 Example Prompt  
> Create a column called plan_region_combo as CONCAT(plan_tier, '_', region).

---

### 🔍 Exploration  
Which plan-region combos have highest churn?


In [ ]:
%%bigquery
SELECT
  *,
  CONCAT(plan_tier, '_', region) AS plan_region_combo
FROM
  `netflix.churn_features`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,region,plan_tier,age_band,total_minutes,avg_rating,churn_label,plan_region_combo
0,user_08194,USA,Premium,18_29,326.9,NaN,0,Premium_USA
1,user_04649,USA,Standard,18_29,595.3,NaN,0,Standard_USA
2,user_08139,Canada,Premium+,18_29,645.0,NaN,0,Premium+_Canada
3,user_01749,USA,Standard,18_29,644.3,NaN,0,Standard_USA
4,user_00612,USA,Standard,18_29,786.1,NaN,0,Standard_USA
5,user_05529,USA,Standard,18_29,1208.8,NaN,1,Standard_USA
6,user_03420,USA,Premium+,18_29,529.7,NaN,0,Premium+_USA
7,user_03877,USA,Standard,18_29,743.5,NaN,0,Standard_USA
8,user_00971,USA,Basic,18_29,523.0,NaN,0,Basic_USA
9,user_03115,Canada,Premium+,18_29,515.6,NaN,0,Premium+_Canada



## Task 5.3: Add Missingness Indicator Flags

**🎯 Goal:** Add binary flags to capture NULL values in age_band and avg_rating.  
**📌 Requirements:** Use IS NULL logic to create new flag columns.

---

### 🧠 Prompt Template  
> Create a new column is_missing_[col_name] that is 1 when column is NULL, else 0.

---

### 👩‍🏫 Example Prompt  
> Add is_missing_age that flags rows where age_band IS NULL.

---

### 🔍 Exploration  
Do missing values correlate with churn?


In [ ]:
%%bigquery
SELECT
  *,
  CASE WHEN age_band IS NULL THEN 1 ELSE 0 END AS is_missing_age_band,
  CASE WHEN avg_rating IS NULL THEN 1 ELSE 0 END AS is_missing_avg_rating
FROM
  `netflix.churn_features`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,region,plan_tier,age_band,total_minutes,avg_rating,churn_label,is_missing_age_band,is_missing_avg_rating
0,user_08194,USA,Premium,18_29,326.9,NaN,0,0,1
1,user_04649,USA,Standard,18_29,595.3,NaN,0,0,1
2,user_08139,Canada,Premium+,18_29,645.0,NaN,0,0,1
3,user_01749,USA,Standard,18_29,644.3,NaN,0,0,1
4,user_00612,USA,Standard,18_29,786.1,NaN,0,0,1
5,user_05529,USA,Standard,18_29,1208.8,NaN,1,0,1
6,user_03420,USA,Premium+,18_29,529.7,NaN,0,0,1
7,user_03877,USA,Standard,18_29,743.5,NaN,0,0,1
8,user_00971,USA,Basic,18_29,523.0,NaN,0,0,1
9,user_03115,Canada,Premium+,18_29,515.6,NaN,0,0,1



## Task 5.5: Assemble Enhanced Feature Table

**🎯 Goal:** Create churn_features_enhanced with all engineered columns.  
**📌 Requirements:** Include all prior features + engineered columns.

---

### 🧠 Prompt Template  
> Generate SQL to create churn_features_enhanced with new columns: watch_time_bucket, plan_region_combo, flag_binge, etc.

---

### 👩‍🏫 Example Prompt  
> Build a new table churn_features_enhanced with all original features + engineered ones.

---

### 🔍 Exploration  
Are row counts stable? Any NULLs introduced?


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `netflix.churn_features_enhanced` AS
SELECT
  user_id,
  region,
  plan_tier,
  age_band,
  total_minutes,
  avg_rating,
  churn_label,
  CASE
    WHEN total_minutes < 100 THEN 'low'
    WHEN total_minutes >= 100 AND total_minutes <= 300 THEN 'medium'
    WHEN total_minutes > 300 THEN 'high'
    ELSE 'unknown'
  END AS watch_time_bucket,
  CASE
    WHEN total_minutes > 500 THEN 1
    ELSE 0
  END AS flag_binge,
  CONCAT(plan_tier, '_', region) AS plan_region_combo,
  CASE WHEN age_band IS NULL THEN 1 ELSE 0 END AS is_missing_age_band,
  CASE WHEN avg_rating IS NULL THEN 1 ELSE 0 END AS is_missing_avg_rating
FROM
  `netflix.churn_features`;

Query is running:   0%|          |

""



## Task 6: Retrain Model on Engineered Features

**🎯 Goal:** Train a logistic regression model using churn_features_enhanced.  
**📌 Requirements:** Use BQML logistic_reg model with new feature columns.

---

### 🧠 Prompt Template  
> Write CREATE MODEL SQL using enhanced features including flags and buckets.

---

### 👩‍🏫 Example Prompt  
> Retrain churn_model_enhanced using watch_time_bucket, flag_binge, plan_region_combo.

---

### 🔍 Exploration  
Does model accuracy improve?


In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `netflix.churn_model_enhanced`
OPTIONS(
  model_type='logistic_reg',
  input_label_cols=['churn_label']
) AS
SELECT
  region,
  plan_tier,
  age_band,
  total_minutes,
  avg_rating,
  watch_time_bucket,
  flag_binge,
  plan_region_combo,
  is_missing_age_band,
  is_missing_avg_rating,
  churn_label
FROM
  `netflix.churn_features_enhanced`;

Query is running:   0%|          |

""



## Task 7: Compare Model Performance

**🎯 Goal:** Compare base model vs enhanced model using ML.EVALUATE.  
**📌 Requirements:** Use same evaluation query for both models.

---

### 🧠 Prompt Template  
> Write a SQL query to evaluate churn_model_enhanced and compare with churn_model.

---

### 👩‍🏫 Example Prompt  
> Compare ML.EVALUATE output from both models side-by-side.

---

### 🔍 Exploration  
Which features made the most difference?


### Evaluation of Base Model (`netflix.churn_model`)

In [ ]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL `netflix.churn_model`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.856655,0.0,0.410758,0.514086


### Evaluation of Enhanced Model (`netflix.churn_model_enhanced`)

In [ ]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL `netflix.churn_model_enhanced`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.849975,0.0,0.423247,0.492029
